In [9]:
sc = SparkContext.getOrCreate()

In [10]:
def parsePoint(line):
    values = [float(x) for x in line.split(' ')]
    print("values",values)
    return LabeledPoint(values[0], values[1:])

In [11]:
data = sc.textFile("pandas.txt")

In [12]:
print("data from txt file",data)

data from txt file pandas.txt MapPartitionsRDD[8] at textFile at <unknown>:0


In [13]:
parsedData = data.map(parsePoint)

In [14]:
print("parsedDate",parsedData)

parsedDate PythonRDD[9] at RDD at PythonRDD.scala:48


In [15]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint

In [16]:
model = SVMWithSGD.train(parsedData, iterations=100)

# Evaluating the model on training data
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))

Training Error = 0.17338458929278083


In [17]:
# Save and load model
# model.save(sc, "pythonSVMWithSGDModel")
# sameModel = SVMModel.load(sc, "pythonSVMWithSGDModel")

In [21]:
data_test = sc.textFile("pandas_test.txt")
parsedData_test = data_test.map(parsePoint)
labelsAndPreds_test = parsedData_test.map(lambda p: (p.label, model.predict(p.features)))
trainErr_test = labelsAndPreds_test.filter(lambda lp: lp[0] != lp[1]).count() / float(parsedData_test.count())
print("Testing Error = " + str(trainErr_test))

Training Error = 0.06261835626814796


In [28]:
TP = labelsAndPreds_test.filter(lambda lp: lp[0] == 1 and lp[1]==1).count() 
FP = labelsAndPreds_test.filter(lambda lp: lp[0] == 0 and lp[1]==1).count() 
FN = labelsAndPreds_test.filter(lambda lp: lp[0] == 1 and lp[1]==0).count() 
TN = labelsAndPreds_test.filter(lambda lp: lp[0] == 0 and lp[1]==0).count() 
allNum = float(parsedData_test.count())

print("FP = "+str(FP))
print("FN = "+str(FN))
print("TN = "+str(TN))
print("TP = "+str(TP))
print("all numbers = "+str(allNum))

FP = 401
FN = 95
TN = 7420
TP = 5
all numbers = 7921.0


In [26]:
# from pyspark.mllib.classification import LogisticRegressionWithLBFGS
# from pyspark.mllib.evaluation import BinaryClassificationMetrics
# from pyspark.mllib.util import MLUtils

# # Several of the methods available in scala are currently missing from pyspark
# # Load training data in LIBSVM format
# # data = MLUtils.loadLibSVMFile(sc, "data/mllib/sample_binary_classification_data.txt")

# # Split data into training (60%) and test (40%)
# # training, test = data.randomSplit([0.6, 0.4], seed=11)
# # training.cache()

# # Run training algorithm to build the model
# # model = LogisticRegressionWithLBFGS.train(training)

# # Compute raw scores on the test set
# predictionAndLabels = parsedData.map(lambda lp: (float(model.predict(lp.features)), lp.label))

# # Instantiate metrics object
# metrics = BinaryClassificationMetrics(predictionAndLabels)

# # Area under precision-recall curve
# print("Area under PR = %s" % metrics.areaUnderPR)

# # Area under ROC curve
# print("Area under ROC = %s" % metrics.areaUnderROC)
# Statistics by class
# labels = data.map(lambda lp: lp.label).distinct().collect()
# for label in sorted(labels):
#     print("Class %s precision = %s" % (label, metrics.precision(label)))
#     print("Class %s recall = %s" % (label, metrics.recall(label)))
#     print("Class %s F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))

# # Weighted stats
# print("Weighted recall = %s" % metrics.weightedRecall)
# print("Weighted precision = %s" % metrics.weightedPrecision)
# print("Weighted F(1) Score = %s" % metrics.weightedFMeasure())
# print("Weighted F(0.5) Score = %s" % metrics.weightedFMeasure(beta=0.5))
# print("Weighted false positive rate = %s" % metrics.weightedFalsePositiveRate)